In [18]:
import random

class World:
    def vacuum(this, dirt_count):
        # grid size (10x10)
        this.n = 10  
                

        #Create grid filled with "c"
        this.map = [["c" for _ in range(this.n)] for _ in range(this.n)]
        
        # Add walls "l"
        for i in range(this.n):
            this.map[0][i] = "l"
            this.map[this.n-1][i] = "l"
            this.map[i][0] = "l"
            this.map[i][this.n-1] = "l"
        
        # Add obstacles "#"
        this.map[3][3] = "#"
        this.map[3][4] = "#"
        this.map[7][6] = "#"
        
        # Add random dirt "*"
        this.dirt = 0
        while this.dirt < dirt_count:
            r = random.randint(1, this.n-2)
            c = random.randint(1, this.n-2)
            if this.map[r][c] == "c":
                this.map[r][c] = "*"
                
                this.dirt += 1

        # Agent start position
        this.row = 1
        this.col = 1
        
        
        # Performance counters
        this.moves     = 0
        this.cleans    = 0
        this.wall_hits = 0


    # Sensors
    def isDirty(this):
        return this.map[this.row][this.col] == "*"
    # Actions 
    def moveRight(this):
        if this.map[this.row][this.col + 1] not in ["l", "#"]:
            this.col += 1
            this.moves += 1
            return True
        this.wall_hits += 1
        return False

    def moveLeft(this):
        if this.map[this.row][this.col - 1] not in ["l", "#"]:
            this.col -= 1
            this.moves += 1
            return True
        this.wall_hits += 1
        return False

    def moveUp(this):
        if this.map[this.row - 1][this.col] not in ["l", "#"]:
            this.row -= 1
            this.moves += 1
            return True
        this.wall_hits += 1
        return False

    def moveDown(this):
        if this.map[this.row + 1][this.col] not in ["l", "#"]:
            this.row += 1
            this.moves += 1
            return True
        this.wall_hits += 1
        return False

    def suck(this):
        this.map[this.row][this.col] = "c"
        this.dirt -= 1
        this.cleans += 1

    def show(this):
        for r in range(this.n):
            display_row = this.map[r][:]
            if r == this.row:
                display_row[this.col] = "A"
            print(" ".join(display_row))

# program test
world = World()
world.vacuum(10)
world.show()

l l l l l l l l l l
l A c c * c c c c l
l * c c * c c c c l
l * * # # c c c c l
l c * c * c c c c l
l c c c * c c c c l
l c c c c c c c c l
l c c c * c # c c l
l c c c c c c * c l
l l l l l l l l l l


In [19]:
def randomMove(): #chooses a random direction other than right to move
    possibleMoves = ["Left", "Up", "Down"]     
    
    while(len(possibleMoves) > 0): #loop until the agent selects a direction that is not blocked or len(possibleMoves) = 0
        nextMove = random.choice(possibleMoves) #random selector
        if(nextMove == "Left"):
            if(world.moveLeft()):
                break #agent moved successfully
            else:
                possibleMoves.remove("Left") #to minimize the number of loops
        elif(nextMove == "Up"):
            if(world.moveUp()):
                break #agent moved successfully
            else:
                possibleMoves.remove("Up") #to minimize the number of loops
        elif(nextMove == "Down"):
            if(world.moveDown()):
                break #agent moved successfully
            else:
                possibleMoves.remove("Down") #to minimize the number of loops

def simpleReflexAgent():
    visited = set() #monitring visited locations to avoid looping
    while(world.dirt > 0):
        location = (world.row, world.col) #make a location as a set
        if location not in visited: #if it is a new location
            visited.add(location)
            if(world.isDirty()):
                world.suck()
                
            if(world.moveRight()): #checks if moving right is possible and successfully done
                continue 

        randomMove() #chooses a random direction if moving right is not possible

print("Room before Cleaning Room")
world.show()

simpleReflexAgent()

print("\nRoom after Cleaning")
world.show()

Room before Cleaning Room
l l l l l l l l l l
l A c c * c c c c l
l * c c * c c c c l
l * * # # c c c c l
l c * c * c c c c l
l c c c * c c c c l
l c c c c c c c c l
l c c c * c # c c l
l c c c c c c * c l
l l l l l l l l l l

Room after Cleaning
l l l l l l l l l l
l c c c c c c c c l
l c c c c A c c c l
l c c # # c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c # c c l
l c c c c c c c c l
l l l l l l l l l l


In [20]:
def createDirectionTable():
    
    table = []
    
    # Create snake pattern for 8x8 inner grid
    for row in range(8):  
        if row % 2 == 0:  
            for _ in range(7):  # Move right 7 times
                table.append("Right")

        else:  # Odd rows: move left
            for _ in range(7):  # Move left 7 times
                table.append("Left")
        
        # Move down after each row 
        if row < 7:
            table.append("Down")
    
    return table


def tryAlternativeMove():
   
    alternatives = ["Right", "Left", "Up", "Down"]
    random.shuffle(alternatives)
    
    for direction in alternatives:
        if direction == "Right" and world.moveRight():
            return True
        elif direction == "Left" and world.moveLeft():
            return True
        elif direction == "Up" and world.moveUp():
            return True
        elif direction == "Down" and world.moveDown():
            return True
    
    return False


def tableDrivenAgent():

    #Table Driven Agent: Uses a predefined lookup table for navigation.
    
    # The direction table
    directionTable = createDirectionTable()
    tableIndex = 0
    maxIterations = 500  
    iterations = 0
    
    while world.dirt > 0 and iterations < maxIterations:
        iterations += 1
        
        # Check and clean if current cell is dirty
        if world.isDirty():
            world.suck()
        
        # Get next direction from table
        if tableIndex < len(directionTable):
            nextDirection = directionTable[tableIndex]
            tableIndex += 1
            
            # Try to move according to table
            moved = False
            if nextDirection == "Right":
                moved = world.moveRight()
            elif nextDirection == "Left":
                moved = world.moveLeft()
            elif nextDirection == "Up":
                moved = world.moveUp()
            elif nextDirection == "Down":
                moved = world.moveDown()
            
            # If table direction failed , try alternatives
            if not moved:
                tryAlternativeMove()
        else:
            #search for remaining dirt randomly
            tryAlternativeMove()
    
    if iterations >= maxIterations:
        print("Reached maximum iterations")


# Test the Table Driven Agent

# Create a new world for testing
world2 = World()
world2.vacuum(10)

print("Room before Cleaning Room")
world2.show()

# Use world2 temporarily for this agent
world = world2
tableDrivenAgent()

print("\nRoom after Cleaning")
world.show()

Room before Cleaning Room
l l l l l l l l l l
l A * c c c * c * l
l c c c c c c c c l
l c * # # c c c c l
l c c c c c c c c l
l c c c c c c * c l
l c c c c c * c c l
l c c c c * # c c l
l * c * c c c c c l
l l l l l l l l l l

Room after Cleaning
l l l l l l l l l l
l c c c c c c c c l
l c c c c c c c c l
l c c # # c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c c c c l
l A c c c c # c c l
l c c c c c c c c l
l l l l l l l l l l


In [21]:
def calculate_score(world):
    # lower score = better performance
    return world.moves + world.wall_hits

def getScore(world):
    print(f"  Total Moves   : {world.moves}")
    print(f"  Cells Cleaned : {world.cleans}")
    print(f"  Wall Hits     : {world.wall_hits}")
    print(f"  Score         : {calculate_score(world)}")
    return calculate_score(world)

# Test Simple Reflex Agent
world1 = World()
world1.vacuum(10)
print("\nBefore cleaning:\n")
world1.show()
world = world1
simpleReflexAgent()
print("\nAfter cleaning:\n")
world1.show()
print("\nSimple Reflex Agent Performance:\n")
score1 = getScore(world1)

print("\n=============================================================================\n")


# Test Table Driven Agent
world2 = World()
world2.vacuum(10)
print("\nBefore cleaning:\n")
world2.show()
world = world2
tableDrivenAgent()
print("\nAfter cleaning:\n")
world2.show()
print("\nTable Driven Agent Performance:\n")
score2 = getScore(world2)

print("\n=============================================================================\n")
# Comparison
print("Simple Reflex Agent  vs  Table Driven Agent")
print(f"Total Moves   : {world1.moves}  vs  {world2.moves}")
print(f"Cells Cleaned : {world1.cleans}  vs  {world2.cleans}")
print(f"Wall Hits     : {world1.wall_hits}  vs  {world2.wall_hits}")
print(f"Score         : {score1}  vs  {score2}\n")

if score1 < score2:
    print("Simple Reflex Agent is better")
elif score2 < score1:
    print("Table Driven Agent is better")
else:
    print("Both agents performed equally")


Before cleaning:

l l l l l l l l l l
l A c c * c c c c l
l c c * c * c c c l
l c c # # * c * c l
l c c c c c c c c l
l c c c * c c * c l
l c c c c * c c c l
l * c c c c # c c l
l c c c c * c c c l
l l l l l l l l l l

After cleaning:

l l l l l l l l l l
l c c c c c c c c l
l c c c c c c c c l
l c c # # c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c c c # c c l
l c c c c c A c c l
l l l l l l l l l l

Simple Reflex Agent Performance:

  Total Moves   : 230
  Cells Cleaned : 10
  Wall Hits     : 50
  Score         : 280



Before cleaning:

l l l l l l l l l l
l A c c c * c c c l
l c c c c c c c c l
l * * # # c c c c l
l c * c c c c c c l
l c * c c c c c c l
l c c c c c c c c l
l c c * * c # c c l
l * c * * c c c c l
l l l l l l l l l l

After cleaning:

l l l l l l l l l l
l c c c c c c c c l
l c c c c c c c c l
l c c # # c c c c l
l c c c c c c c c l
l c c c c c c c c l
l c c c A c c c c l
l c c c c c # c c l
l c c c c c c c c l
l l l l l l l l l l

Ta